In [1]:
# Setup
import sympy as sym
import numpy as np
sym.init_printing()
%pylab inline


Populating the interactive namespace from numpy and matplotlib


## Question 2
*Propagate the uncertainty in measurement*

Define the function $f(x,y)=\log(\frac{x}{\sqrt{x^2+y^2}}) \sqrt{10+cos(x)}$ symbolically using *SymPy*.

There error of $f(x,y)$, $\sigma_f$ is given by $\sigma_f=\sqrt{(\frac{\partial f}{\partial x} \sigma_x)^2 + (\frac{\partial f}{\partial y} \sigma_y)^2}$

In [2]:
x, y, sigma_x, sigma_y = sym.symbols('x y sigma_x sigma_y')

F = sym.log(x/sym.sqrt(x**2+y**2))*sym.sqrt(10+sym.cos(x))

sigma_f = sym.sqrt((sym.diff(F, x)*sigma_x )**2 +
                   (sym.diff(F,y)*sigma_y)**2)


Now we can evaluate our function and calculate its error on the range of values below:

In [4]:
values = [
    {'x':1.0, 'sigma_x':0.2, 'y':2.3, 'sigma_y':0.1},
    {'x':1.2, 'sigma_x':0.1, 'y':3.0, 'sigma_y':.2},
    {'x':2.1, 'sigma_x':0.7, 'y':1.0, 'sigma_y':.7},
    {'x':3.2, 'sigma_x':0.4, 'y':4.1, 'sigma_y':0.6},
]

# calculate the values and errors on f
for subs in values:
    val = F.evalf(subs=subs)
    err = sigma_f.evalf(subs=subs)
    print( 
       ('f({x:.1f}±{sigma_x:.1f}, {y:.1f}±{sigma_y:.1f}) = '+
        '{val: 0.1f} ± {err:0.1f}').format(val=val, err=err, **subs)
    )

f(1.0±0.2, 2.3±0.1) = -3.0 ± 0.6
f(1.2±0.1, 3.0±0.2) = -3.2 ± 0.3
f(2.1±0.7, 1.0±0.7) = -0.3 ± 0.4
f(3.2±0.4, 4.1±0.6) = -1.5 ± 0.4


## Question 3


In [113]:
# near linear model y(x)=a+bx
# 

data = [
    #  x,      y,  sigma_y
    [ 0.00,  4.92, 0.86],
    [ 0.53,  5.45, 1.19],
    [ 1.05,  5.89, 1.29],
    [ 1.58,  8.12, 0.12],
    [ 2.11,  9.20, 0.61],
    [ 2.63, 10.86, 0.62],
    [ 3.16, 10.33, 1.26],
    [ 3.68, 15.28, 1.42],
    [ 4.21, 13.80, 1.07],
    [ 4.74, 13.86, 0.44],
    [ 5.26, 14.57, 0.72],
    [ 5.79, 15.48, 0.65],
    [ 6.32, 17.91, 0.72],
    [ 6.84, 17.83, 0.65],
    [ 7.37, 21.25, 1.64],
    [ 7.89, 21.72, 0.59],
    [ 8.42, 21.80, 0.05],
    [ 8.95, 23.52, 1.45],
    [ 9.47, 25.47, 1.25],
    [10.00, 24.95, 0.05],
]
# Split into vectors
X, Y, sigma_Y = map(np.array, zip(*data))

A) Evaluate the mean, standard deviation, and error on the mean of Y

In [127]:
Y_mean  = Y.mean()
Y_stdev = Y.std(ddof=1)
Y_mean_err = Y_stdev/sqrt(len(Y)-1)

print('Y_mean  = {:.2f} ± {:.1f}'.format(Y_mean, Y_mean_err))
print('Y_stdev = {:.2f}'.format(Y_stdev))

Y_mean  = 15.11 ± 1.5
Y_stdev = 6.60


B) Fit the model